## Exporting Models

In order to make a tensorflow model usable by other systems, tensorflow provides multiple different methods to export a model. For this use case we will be defining a subgraph of the original model.

We will then produce a final "frozen" graph which is an optimized version of an original model

### Importing

Below we import all out utilities as well as cells from the build and train notebook

In [2]:
import os, argparse
import tensorflow as tf
from tensorflow.python.framework import graph_util

import tensorflow as tf;tf.reset_default_graph()
import numpy as np
import argparse
import os
import json
import glob
import random
import collections
import math
import time

from util import run_cells

In [4]:
run_cells("./BuildingAndTrainModels.ipynb", 
          cell_tags=["parameters","conv","gen","disc","model","util","process"])

# Reducing the model

Before we can export the model as a single file, we must reduce the model into a smaller subgraph containing just the generator.

In [12]:
def create_single_generator():

    ## Create model inputs
    input = tf.placeholder(tf.string, shape=[1])
    input_image = tf.image.decode_png(tf.decode_base64(input[0]))

    # remove alpha channel if present
    input_image = tf.cond(tf.equal(tf.shape(input_image)[2], 4), lambda: input_image[:,:,:3], lambda: input_image)
    # convert grayscale to RGB
    input_image = tf.cond(tf.equal(tf.shape(input_image)[2], 1), lambda: tf.image.grayscale_to_rgb(input_image), lambda: input_image)

    ## Crop and batch input
    input_image = tf.image.convert_image_dtype(input_image, dtype=tf.float32)
    input_image.set_shape([CROP_SIZE, CROP_SIZE, 3])
    batch_input = tf.expand_dims(input_image, axis=0)

    ## Create generator model
    with tf.variable_scope("generator"):
        batch_output = deprocess(create_generator(preprocess(batch_input), 3))

    ## Convert model output to filetype output
    output_filetype = "png"
    output_image = tf.image.convert_image_dtype(batch_output, dtype=tf.uint8)[0]
    if output_filetype == "png":
        output_data = tf.image.encode_png(output_image)
    elif output_filetype == "jpeg":
        output_data = tf.image.encode_jpeg(output_image, quality=80)
    else:
        raise Exception("invalid filetype")
    output = tf.convert_to_tensor([tf.encode_base64(output_data)])
    
    return input, output

Using a helper function we can produce our inputs and outputs for this respective model

In [13]:
input, output = create_single_generator()

Now we can add the input and output tensors to the graph collection that tensorflow is aware of

We will then use this collected ops in the graph to load our previously saved model, then after loading it, we will newly export it to a file containing only the weights we care about transfered to this new subgraph

In [14]:
key = tf.placeholder(tf.string, shape=[1])
inputs = {
    "key": key.name,
    "input": input.name
}
tf.add_to_collection("inputs", json.dumps(inputs))
outputs = {
    "key":  tf.identity(key).name,
    "output": output.name,
}
tf.add_to_collection("outputs", json.dumps(outputs))

## Reducing and Exporting

In the next cell we will read in a tensorflow checkpoint, then exporting the graph to a file

In [16]:
init_op = tf.global_variables_initializer()
restore_saver = tf.train.Saver()
export_saver = tf.train.Saver()

input_checkpoint_dir = "./models/face2face-model/"
output_model_dir = "./models/face2face-reduced-model/"

with tf.Session() as sess:

    sess.run(init_op)
    
    print("Loading model from checkpoint ...")
    latest_checkpoint = tf.train.latest_checkpoint(input_checkpoint_dir)
    restore_saver.restore(sess, latest_checkpoint)
    
    print("Exporting model ...")
    export_saver.export_meta_graph(filename=os.path.join(output_model_dir, "export.meta"))
    export_saver.save(sess, os.path.join(output_model_dir, "export"), write_meta_graph=False)

Loading model from checkpoint ...
INFO:tensorflow:Restoring parameters from ./models/face2face-model/model-32000
Exporting model ...


In [73]:
def process_image(x):
    with tf.name_scope('load_images'):
        raw_input = tf.image.convert_image_dtype(x, dtype=tf.float32)

        raw_input.set_shape([None, None, 3])

        # break apart image pair and move to range [-1, 1]
        width = tf.shape(raw_input)[1]  # [height, width, channels]
        a_images = preprocess(raw_input[:, :width // 2, :])
        b_images = preprocess(raw_input[:, width // 2:, :])

    inputs, targets = [a_images, b_images]

    # synchronize seed for image operations so that we do the same operations to both
    # input and output images
    def transform(image):
        # area produces a nice downscaling, but does nearest neighbor for upscaling
        # assume we're going to be doing downscaling here
        return tf.image.resize_images(image, [CROP_SIZE, CROP_SIZE], method=tf.image.ResizeMethod.AREA)

    with tf.name_scope('input_images'):
        input_images = tf.expand_dims(transform(inputs), 0)

    with tf.name_scope('target_images'):
        target_images = tf.expand_dims(transform(targets), 0)

    return input_images, target_images

def create_model(inputs, targets):
    with tf.variable_scope('generator') as scope:
        out_channels = int(targets.get_shape()[-1])
        outputs = create_generator(inputs, out_channels)
    return outputs

def convert(image):
    return tf.image.convert_image_dtype(image, dtype=tf.uint8, saturate=True, name='output')  # output tensor

def generate_output(x):
    with tf.name_scope('generate_output'):
        test_inputs, test_targets = process_image(x)

        # inputs and targets are [batch_size, height, width, channels]
        model = create_model(test_inputs, test_targets)

        # deprocess files
        outputs = deprocess(model)

        # reverse any processing on images so they can be written to disk or displayed to user
        converted_outputs = convert(outputs)
    return converted_outputs

tf.reset_default_graph()

input_folder = "./models/face2face-model/"
output_folder = "./models/face2face-reduced-model/"

a = tf.placeholder(tf.uint8, shape=(256, 512, 3), name='image_tensor')  # input tensor
y = generate_output(a)

with tf.Session() as sess:
    # Restore original model
    saver = tf.train.Saver()
    checkpoint = tf.train.latest_checkpoint(input_folder)
    print (checkpoint)
    saver.restore(sess, checkpoint)

    # Export reduced model used for prediction
    saver = tf.train.Saver()
    saver.save(sess, '{}/reduced_model'.format(output_folder))
    print("Model is exported to {}".format(checkpoint))

./models/face2face-model/export
INFO:tensorflow:Restoring parameters from ./models/face2face-model/export
Model is exported to ./models/face2face-model/export


# Freezing the model

Now that we have a reduced form of our model we will now "freeze" it by exporting the tensorflow checkpoint to a frozen protobuff

In [74]:
# We retrieve our checkpoint fullpath
checkpoint = tf.train.get_checkpoint_state(output_folder)
input_checkpoint = checkpoint.model_checkpoint_path

In [75]:
# We precise the file fullname of our freezed graph
absolute_model_folder = '/'.join(input_checkpoint.split('/')[:-1])
output_graph = output_folder + '/frozen_model.pb'

In [76]:
# Before exporting our graph, we need to precise what is our output node
# This is how TF decides what part of the Graph he has to keep and what part it can dump
# NOTE: this variable is plural, because you can have multiple output nodes
output_node_names = 'generate_output/output'

In [77]:
# We clear devices to allow TensorFlow to control on which device it will load operations
clear_devices = True

In [78]:
# We import the meta graph and retrieve a Saver
saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)

In [79]:
# We retrieve the protobuf graph definition
graph = tf.get_default_graph()
input_graph_def = graph.as_graph_def()

# We start a session and restore the graph weights
with tf.Session() as sess:
    saver.restore(sess, input_checkpoint)

    # We use a built-in TF helper to export variables to constants
    output_graph_def = graph_util.convert_variables_to_constants(
        sess,  # The session is used to retrieve the weights
        input_graph_def,  # The graph_def is used to retrieve the nodes
        output_node_names.split(",")  # The output node names are used to select the usefull nodes
    )

    # Finally we serialize and dump the output graph to the filesystem
    with tf.gfile.GFile(output_graph, 'wb') as f:
        f.write(output_graph_def.SerializeToString())
    print('%d ops in the final graph.' % len(output_graph_def.node))

INFO:tensorflow:Restoring parameters from ./models/face2face-reduced-model/reduced_model
INFO:tensorflow:Froze 76 variables.
Converted 76 variables to const ops.
405 ops in the final graph.
